In [2]:
folder = '/sciclone/scr10/dchendrickson01/EFIT/Double4m/'
file = 'Anima.csv'


In [3]:
import pandas as pd
import dask.dataframe as dd
import numpy as np

In [4]:
# this is needed to call dask.compute
dataset= dd.read_csv(folder+file, low_memory=False) #, dtype={'time':'object',
#                                                           'x':'object',
#                                                           'y':'object',
#                                                           'z':'object',
#                                                           'Energy':'object',
#                                                           'DisX':'object',
#                                                           'DisY':'object',
#                                                           'DisZ':'object',
#                                                          })

In [5]:
dataset.head

<bound method _Frame.head of Dask DataFrame Structure:
                   time      x      y      z   Energy     DisX     DisY     DisZ
npartitions=1027                                                                
                  int64  int64  int64  int64  float64  float64  float64  float64
                    ...    ...    ...    ...      ...      ...      ...      ...
...                 ...    ...    ...    ...      ...      ...      ...      ...
                    ...    ...    ...    ...      ...      ...      ...      ...
                    ...    ...    ...    ...      ...      ...      ...      ...
Dask Name: read-csv, 1027 tasks>

In [6]:
dataset['x'].max().compute()

ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+--------+--------+----------+
| Column | Found  | Expected |
+--------+--------+----------+
| x      | object | int64    |
+--------+--------+----------+

The following columns also raised exceptions on conversion:

- x
  ValueError("invalid literal for int() with base 10: 'x'")

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'x': 'object'}

to the call to `read_csv`/`read_table`.

In [ ]:
dataset['time'].max().compute()

In [ ]:
# for example take a subset
dataset = dataset[dataset['time']==38990].compute()

In [37]:
maxDim = 977 #np.max([dataset['x'].max(), dataset['y'].max(), dataset['z'].max()])

In [38]:
maxDim+=1
maxDim = int(maxDim)

In [40]:
dataset.compute()

,time,x,y,z,Energy,DisX,DisY,DisZ
0,100000,0,1,6,0.0,0.0,0.0,0.0
1,100000,0,2,6,0.0,0.0,0.0,0.0
2,100000,0,3,6,0.0,0.0,0.0,0.0
3,100000,0,4,6,0.0,0.0,0.0,0.0
4,100000,0,5,6,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
205717,100000,799,27,2,0.0,0.0,0.0,0.0
205718,100000,799,27,3,0.0,0.0,0.0,0.0
205719,100000,799,27,4,0.0,0.0,0.0,0.0
205720,100000,799,27,5,0.0,0.0,0.0,0.0


In [41]:
print('have data')

Max = dataset['Energy'].max()
Min = dataset['Energy'].min()

print(Min, Max)

have data
dd.Scalar<series-..., dtype=float64> dd.Scalar<series-..., dtype=float64>


In [14]:
dataset['Energy'] -= Min
dataset['Energy'] += .00000001

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:
buckets = 15
bins = np.linspace(0.00000001,Max+0.00000001,buckets+1)

import matplotlib.cm as colors

cmap = colors.get_cmap('rainbow')

colorBin = np.linspace(0,1,buckets)
Colors = []

for cb in colorBin:
    Colors.append(cmap(cb))

In [ ]:
counts, bins = np.histogram(dataset['Energy'],bins=bins)

In [ ]:
counts

In [ ]:

data = np.empty((maxDim,int(maxDim/2),int(maxDim/2)))
color = np.empty((maxDim,int(maxDim/2),int(maxDim/2),4))
i=0
for part in dataset.partitions:
    for item in part.iterrows():
        data[int(item[1][1]),int(item[1][2]),int(item[1][3])] = item[1][4]
        color[int(item[1][1]),int(item[1][2]),int(item[1][3]),:] = Colors[min(range(len(colorBin)), key=lambda i: abs(colorBin[i]-item[1][4]))]
        i+=1
        
print('image data made')

In [ ]:
import matplotlib.pyplot as plt

# Define dimensions
Nx, Ny, Nz = maxDim, int(maxDim/2), int(maxDim/2)
#X, Y, Z = np.meshgrid(np.arange(Nx), np.arange(Ny), -np.arange(Nz))

# Create a figure with 3D ax
fig = plt.figure(figsize=(12,12), dpi=100)
ax = fig.add_subplot(111, projection='3d')

print('fig set up')

data = data !=0

print('data made true/false')
ax.voxels(data,
          facecolors=color,
          linewidth=0.01)
#ax.set(xlabel='r', ylabel='g', zlabel='b')
#ax.set_aspect('equal')
# 

print('Axis set up')

# Show Figure
plt.savefig(folder+'singleimage4.png')
plt.show()

In [ ]:
asdfasdf

In [ ]:
"""
===============================
3D voxel plot of the NumPy logo
===============================

Demonstrates using `.Axes3D.voxels` with uneven coordinates.
"""

import matplotlib.pyplot as plt
import numpy as np


def explode(data):
    size = np.array(data.shape)*2
    data_e = np.zeros(size - 1, dtype=data.dtype)
    data_e[::2, ::2, ::2] = data
    return data_e

# build up the numpy logo
n_voxels = np.zeros((4, 3, 4), dtype=bool)
n_voxels[0, 0, :] = True
n_voxels[-1, 0, :] = True
n_voxels[1, 0, 2] = True
n_voxels[2, 0, 1] = True
facecolors = np.where(n_voxels, '#FFD65DC0', '#7A88CCC0')
edgecolors = np.where(n_voxels, '#BFAB6E', '#7D84A6')
filled = np.ones(n_voxels.shape)

# upscale the above voxel image, leaving gaps
filled_2 = explode(filled)
fcolors_2 = explode(facecolors)
ecolors_2 = explode(edgecolors)

# Shrink the gaps
x, y, z = np.indices(np.array(filled_2.shape) + 1).astype(float) // 2
x[0::2, :, :] += 0.05
y[:, 0::2, :] += 0.05
z[:, :, 0::2] += 0.05
x[1::2, :, :] += 0.95
y[:, 1::2, :] += 0.95
z[:, :, 1::2] += 0.95

ax = plt.figure().add_subplot(projection='3d')
ax.voxels(x, y, z, filled_2, facecolors=fcolors_2, edgecolors=ecolors_2)
ax.set_aspect('equal')

plt.show()


In [ ]:
"""
=======================================================
3D voxel / volumetric plot with cylindrical coordinates
=======================================================

Demonstrates using the *x*, *y*, *z* parameters of `.Axes3D.voxels`.
"""

import matplotlib.pyplot as plt
import numpy as np

import matplotlib.colors


def midpoints(x):
    sl = ()
    for i in range(x.ndim):
        x = (x[sl + np.index_exp[:-1]] + x[sl + np.index_exp[1:]]) / 2.0
        sl += np.index_exp[:]
    return x

# prepare some coordinates, and attach rgb values to each
r, theta, z = np.mgrid[0:1:11j, 0:np.pi*2:25j, -0.5:0.5:11j]
x = r*np.cos(theta)
y = r*np.sin(theta)

rc, thetac, zc = midpoints(r), midpoints(theta), midpoints(z)

# define a wobbly torus about [0.7, *, 0]
sphere = (rc - 0.7)**2 + (zc + 0.2*np.cos(thetac*2))**2 < 0.2**2

# combine the color components
hsv = np.zeros(sphere.shape + (3,))
hsv[..., 0] = thetac / (np.pi*2)
hsv[..., 1] = rc
hsv[..., 2] = zc + 0.5
colors = matplotlib.colors.hsv_to_rgb(hsv)

# and plot everything
ax = plt.figure().add_subplot(projection='3d')
ax.voxels(x, y, z, sphere,
          facecolors=colors,
          edgecolors=np.clip(2*colors - 0.5, 0, 1),  # brighter
          linewidth=0.5)

plt.show()


In [ ]:
buckets = 15
#bins = np.linspace(1,Min+Max+1,buckets+1)

import matplotlib.cm as colors

cmap = colors.get_cmap('rainbow')

colorBin = np.linspace(0,1,buckets)
Colors = []

for cb in colorBin:
    Colors.append(cmap(cb))

In [ ]:
Colors